In [1]:
import os
import sys
import csv
import pandas as pd
import numpy as np

import os.path
from os import path

import PIL
from PIL import Image

import cv2
import glob
import h5py
import os
import pathlib
import tables

import torch
from skimage import io, transform
from sklearn import metrics
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from matplotlib import pyplot

import torch
import timm
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import torch.nn.functional as F
import torch.nn as nn
from torch.nn.functional import kl_div, softmax, log_softmax
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import seaborn as sns

/home/irene/myenv/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
hdf5_path = '/home/irene/ISEF-Project-2023/DensityHDF51.hdf5'
file = h5py.File(hdf5_path, "r")
file.keys()

<KeysViewHDF5 ['test_img', 'test_label', 'test_path', 'train_img', 'train_label', 'train_path', 'val_img', 'val_label', 'val_path']>

In [3]:
rgb_mean,rgb_std = (0.485, 0.456, 0.406),(0.229, 0.224, 0.225)

In [4]:
class HDF5Dataset(Dataset):


    def __init__(self, h5_path, set_name, transform = None):
        self.h5_path = h5_path
        self.dataset = None
        self.transform = transform
        self.file_path  = h5_path
        self.set = set_name
        
        str_name = self.set + "_img"
        
        file = h5py.File(h5_path, "r")
        self.dataset_len = len(file[str_name])
        
        self.transform = transforms.Compose([
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.Resize(224),
                transforms.ToTensor(),
                transforms.Normalize(rgb_mean,rgb_std)
            ])
        
    def __getitem__(self, index): #to enable indexing
        
        if self.dataset is None:
            self.imgs = h5py.File(self.file_path, 'r')[self.set + "_img"]
            self.labels = h5py.File(self.file_path, 'r')[self.set + "_label"]
            
            cur_img = self.imgs[index]
            PIL_image = Image.fromarray(np.uint8(cur_img)).convert('RGB')#3 channels don't need alpha channel network input
            image = self.transform(PIL_image)
            label = self.labels[index].astype('float32')
            
            
        return (image,label)

    def __len__(self):
        return self.dataset_len

In [5]:
from torch.utils.data import Dataset,DataLoader

train_loader = DataLoader(HDF5Dataset(h5_path=hdf5_path,set_name="train"), batch_size=8,shuffle=True,drop_last=False)
val_loader = DataLoader(HDF5Dataset(h5_path=hdf5_path,set_name="val"), batch_size=8,shuffle=True,drop_last=False)
test_loader = DataLoader(HDF5Dataset(h5_path=hdf5_path,set_name="test"), batch_size=8,shuffle=True,drop_last=False)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#out_dim = 2

model = timm.create_model("resnet50", pretrained=True)
#model.fc = nn.Linear(model.fc.in_features, out_dim)

model.to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.001,weight_decay = 1e-4,momentum = 0.9)
criterion = nn.CrossEntropyLoss()


#epoch 20 --> epoch 100 or 50 see what happens
def train_valid_model (net,loaders,max_epochs = 20):
    best_acc = 0.0 
    for epoch in range (max_epochs):
        for phase in ['train','val']:
            iterator = iter(loaders[phase])
            total_step = len(loaders[phase])
            print('Phase {}'.format(phase))
            if phase == 'train':
                net.train()
            else:
                net.eval()
                
            running_loss = 0.0
            running_correct = 0
            
            predictions_all = []
            label_all = []
            probs_all = []
            for step in range(total_step-1): #iterate each batch
                images,labels = next(iterator) # CUDA computation
                images = images.to(device)
                labels = labels.to(device)
                labels = labels.long()
                optimizer.zero_grad()
                
                output = net(images)
                loss = criterion(output,labels)
                
                probs = torch.nn.functional.softmax(output, dim=1) # probabilities
                
                running_loss +=loss.item()
                _, preds = torch.max(output.data,1)
                
                running_correct += (preds == labels).sum().item()
                
                if len(predictions_all) == 0:
                    predictions_all = preds.detach().cpu().numpy()
                    label_all = labels.detach().cpu().numpy()
                    probs_all = probs.detach().cpu().numpy()
                else:
                    predictions_all = np.vstack((predictions_all, preds.detach().cpu().numpy()))
                    probs_all = np.vstack((probs_all, probs.detach().cpu().numpy()))
                    label_all = np.vstack((label_all, labels.detach().cpu().numpy()))

                if phase == 'train':
                    loss.backward()
                    optimizer.step()
            
                    
            phase_loss = running_loss / len(loaders[phase])
            phase_acc = running_correct/len(label_all.flatten())
            if phase == 'val':
                y_true = label_all.flatten()
                y_pred = predictions_all.flatten()
                print("validating...")
                print(len(y_true))
                print(len(y_pred))
                print(metrics.classification_report(y_true, y_pred, digits=3))
            if phase == 'val' and phase_acc > best_acc:
                
                best_acc = phase_acc
                import copy 
                
                best_model_state_dict = copy.deepcopy(net.state_dict())
                torch.save(best_model_state_dict,'densityresnet50best_model.pth')
                
            print('PHASE {} Loss: {:.4f} Acc: {:.4f}'.format(phase, phase_loss, phase_acc))
    net.load_state_dict(best_model_state_dict)
    
    return net 
            
       
print(device)  

cuda


In [7]:
def test_best_model (net, test_loader, a_device = None):
    iterator = iter(test_loader)
    total_step = len(test_loader)
    
    print(total_step)
    net.eval()
    
    with torch.no_grad():
        running_loss = 0
        running_corrects = 0
        
        total_0,total_1 = 0,0
        hit_0 = 0
        hit_1 = 0
        label_all = []
        probs_all = []
        predictions_all = []
        for step in range(total_step-1):
            images,labels = next(iterator)
            images.to(a_device)
            labels.to(a_device)
            total_0 += labels.tolist().count(0)
            total_1 += labels.tolist().count(1)
            print(labels.shape)
            images = images.to(a_device)
            labels = labels.to(device=a_device, dtype=torch.int64)
            labels = labels.long()
            
            output = net(images)
            loss = criterion(output,labels)
            
            probs = torch.nn.functional.softmax(output, dim=1)

            running_loss +=loss.item()
            _, preds = torch.max(output.data,1)
            
            equals = preds == labels.view(*preds.shape)
            if(len(label_all) ==0):
                predictions_all = preds.detach().cpu().numpy()
                label_all = labels.detach().cpu().numpy()
                probs_all = probs.detach().cpu().numpy()
            else:
                predictions_all = np.vstack((predictions_all, preds.detach().cpu().numpy()))
                label_all = np.vstack((label_all, labels.detach().cpu().numpy()))
                probs_all = np.vstack((probs_all, probs.detach().cpu().numpy()))

            all_hits = equals.view(equals.shape[0]).tolist() 
            all_corrects = labels[all_hits]
            
            hit_0 += all_corrects.tolist().count(0)
            hit_1 += all_corrects.tolist().count(1)
 
        
            running_corrects += (preds == labels).sum().item()
        print(len(label_all.flatten()))
        label_all = label_all.flatten()
        predictions_all = predictions_all.flatten()
        phase_loss = running_loss / len(test_loader)
        phase_acc = running_corrects/len(label_all.flatten())
        print('Test Loss: {:.4f} Acc: {:.4f}'.format(phase_loss, phase_acc))
        
        print(hit_0, ' / ',total_0)
        print(hit_1, ' / ',total_1)
                
            
    return label_all, probs_all, predictions_all #add this later
                
        #y_test --> label, y_score --> probs all 

In [8]:
epochs = 100
loaders = dict({'train': train_loader, 'val': val_loader})
for t in range(epochs):
    print(f"Epoch {t+1}\n-----------------------")
    bestmodel = train_valid_model (model,loaders)
    

Epoch 1
-----------------------
Phase train
PHASE train Loss: 5.6032 Acc: 0.1354
Phase val
validating...
160
160
              precision    recall  f1-score   support

           0      0.578     0.430     0.493        86
           1      0.512     0.333     0.404        66
           2      0.000     0.000     0.000         6
           3      0.000     0.000     0.000         2
           4      0.000     0.000     0.000         0
          15      0.000     0.000     0.000         0
          16      0.000     0.000     0.000         0
          35      0.000     0.000     0.000         0
          49      0.000     0.000     0.000         0
          59      0.000     0.000     0.000         0
         130      0.000     0.000     0.000         0
         139      0.000     0.000     0.000         0
         151      0.000     0.000     0.000         0
         153      0.000     0.000     0.000         0
         172      0.000     0.000     0.000         0
         225      0.00

/home/irene/myenv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/irene/myenv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/irene/myenv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/irene/myenv/lib/python3.6/site-packages/sklearn/

PHASE val Loss: 3.3309 Acc: 0.3688
Phase train
PHASE train Loss: 1.9210 Acc: 0.4875
Phase val
validating...
160
160
              precision    recall  f1-score   support

           0      0.573     0.547     0.560        86
           1      0.486     0.515     0.500        66
           2      0.000     0.000     0.000         6
           3      0.000     0.000     0.000         2
         129      0.000     0.000     0.000         0
         501      0.000     0.000     0.000         0
         523      0.000     0.000     0.000         0
         538      0.000     0.000     0.000         0
         648      0.000     0.000     0.000         0
         822      0.000     0.000     0.000         0
         845      0.000     0.000     0.000         0
         982      0.000     0.000     0.000         0

    accuracy                          0.506       160
   macro avg      0.088     0.088     0.088       160
weighted avg      0.508     0.506     0.507       160



/home/irene/myenv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/irene/myenv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/irene/myenv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/irene/myenv/lib/python3.6/site-packages/sklearn/

PHASE val Loss: 1.7371 Acc: 0.5062
Phase train
PHASE train Loss: 1.3560 Acc: 0.5312
Phase val
validating...
160
160
              precision    recall  f1-score   support

           0      0.613     0.535     0.571        86
           1      0.488     0.606     0.541        66
           2      0.000     0.000     0.000         6
           3      0.000     0.000     0.000         2
         459      0.000     0.000     0.000         0
         813      0.000     0.000     0.000         0
         974      0.000     0.000     0.000         0

    accuracy                          0.537       160
   macro avg      0.157     0.163     0.159       160
weighted avg      0.531     0.537     0.530       160



...

PHASE train Loss: 0.0067 Acc: 0.9979
Phase val
validating...
160
160
              precision    recall  f1-score   support

           0      0.676     0.812     0.738        85
           1      0.691     0.567     0.623        67
           2      0.333     0.167     0.222         6
           3      0.000     0.000     0.000         2

    accuracy                          0.675       160
   macro avg      0.425     0.386     0.396       160
weighted avg      0.661     0.675     0.661       160

PHASE val Loss: 1.4141 Acc: 0.6750
Phase train


/home/irene/myenv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/irene/myenv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/irene/myenv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


PHASE train Loss: 0.0058 Acc: 0.9979
Phase val
validating...
160
160
              precision    recall  f1-score   support

           0      0.755     0.835     0.793        85
           1      0.734     0.701     0.718        67
           2      0.500     0.167     0.250         6
           3      0.000     0.000     0.000         2

    accuracy                          0.744       160
   macro avg      0.497     0.426     0.440       160
weighted avg      0.728     0.744     0.731       160



/home/irene/myenv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/irene/myenv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/irene/myenv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


PHASE val Loss: 1.4181 Acc: 0.7438


In [8]:
model_path = 'densityresnet50best_model.pth'

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = timm.create_model("resnet50", pretrained=True)


In [19]:
my_dict = torch.load(model_path)

In [20]:
model.load_state_dict(my_dict)

<All keys matched successfully>

In [21]:
model.to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.001,weight_decay = 1e-4,momentum = 0.9)
criterion = nn.CrossEntropyLoss()

In [22]:
y_test, y_prob, y_pred= test_best_model (model, test_loader, a_device = device)


21
torch.Size([8])
torch.Size([8])
torch.Size([8])
torch.Size([8])
torch.Size([8])
torch.Size([8])
torch.Size([8])
torch.Size([8])
torch.Size([8])
torch.Size([8])
torch.Size([8])
torch.Size([8])
torch.Size([8])
torch.Size([8])
torch.Size([8])
torch.Size([8])
torch.Size([8])
torch.Size([8])
torch.Size([8])
torch.Size([8])
160
Test Loss: 2.2533 Acc: 0.6562
83  /  91
20  /  46


In [25]:
y_test

array([ 0,  0,  1,  0,  1,  0,  0,  1,  0,  0,  0,  1,  0,  1,  0,  0,  2,
        1,  1,  0,  0, 10,  4,  1,  0,  1,  1,  1,  1,  0,  1,  0,  1,  0,
        4,  0,  1,  0,  6,  0,  6,  0,  0,  0,  0,  0,  0,  0,  1,  0,  1,
        1,  1,  5,  0,  2,  0,  0,  0,  2,  0,  0,  0,  0,  2,  0,  0,  0,
        6,  0,  0,  0,  2,  0,  0,  1,  1,  1,  1,  1,  0,  1,  1,  0,  2,
        3,  0,  0,  3,  1,  0,  1,  7,  0,  1,  0,  1,  0,  0,  1,  0,  0,
        0,  1,  0,  1,  1,  2,  4,  1,  0,  9,  0,  0,  0,  1,  0,  0,  5,
        0,  0,  0,  0,  1,  0,  0,  1,  1,  0,  0,  2,  0,  1,  3,  0,  0,
        0,  0,  0,  0,  0,  1,  1,  0,  0,  0,  0,  1,  0,  1,  0,  1,  0,
        0,  1,  0,  0,  0,  0,  2])

In [26]:
y_pred

array([0, 8, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 4, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 2, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 2, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 2, 1, 1,
       0, 2, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 0,
       0, 3, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 4, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])